# Load dependencies

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score

import optuna.integration.lightgbm as lgb
from lightgbm import LGBMClassifier

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
from optuna.integration import LightGBMPruningCallback

from tqdm import tqdm

In [ ]:
train=pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
test=pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')
sub=pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')

In [ ]:
conditions = [
    (train.target == "Class_1"), (train.target == "Class_2"), (train.target == "Class_3"),
    (train.target == "Class_4"), (train.target == "Class_5"), (train.target == "Class_6"),
    (train.target == "Class_7"), (train.target == "Class_8"), (train.target == "Class_9")
]
choices = [0, 1, 2, 3, 4, 5, 6, 7, 8]
train["target"] = np.select(conditions, choices)

In [ ]:
X_test = test.drop(['id'], axis=1)
X = train.drop(['id', 'target'], axis=1)
y = train.target

# LightGBMTunerCV

In [ ]:
dtrain = lgb.Dataset(X, label=y)

In [ ]:
params = {
    "objective": "multiclass",
    "num_class": 9,
    "metric": "multi_logloss",
    "verbosity": -1,
    "boosting_type": "gbdt",
    'learning_rate': 0.03,
    'random_state': 314
    }

In [ ]:
tuner = lgb.LightGBMTunerCV(
    params, dtrain, verbose_eval=100, early_stopping_rounds=100, 
    folds=StratifiedKFold(n_splits=4, shuffle=True)
)

tuner.run()

In [ ]:
tuner.best_score

In [ ]:
tuner.best_params

# Final model

In [ ]:
final_lgb = LGBMClassifier(**dict({'n_estimators': 500}, **tuner.best_params))

In [ ]:
final_lgb.fit(X, y)
y_pred = final_lgb.predict_proba(test.drop('id', axis=1))

# Sub

In [ ]:
sub.iloc[:, 1:] = y_pred

sub.to_csv('lgbm_tuner.csv', index=False)